In [ ]:
# ============================================================
# FASE 6 - Robustez y validez externa
# Interno:  /content/drive/MyDrive/Strokesdataset.csv
# Externo:  /content/drive/MyDrive/synthetic_stroke_dataset.csv
# Modelo:   rf_randomcv.pkl (ya entrenado)
# Var sensible: work_type (Self-employed)
# ============================================================

import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

from IPython.display import display

from sklearn.metrics import (
    confusion_matrix, balanced_accuracy_score, matthews_corrcoef,
    average_precision_score
)

import joblib

Mounted at /content/drive


In [ ]:
# ============================================================
# 1. Cargar modelo entrenado e identificar features
# ============================================================

# Cargar modelo final
ruta_modelo = '/content/drive/MyDrive/modelos_largos/rf/StrokeSmote/rf_randomcv.pkl'
rf_randomcv = joblib.load(ruta_modelo)

print("Modelo cargado desde:", ruta_modelo)
print("Tipo de modelo:", type(rf_randomcv))

Modelo cargado desde: /content/drive/MyDrive/modelos_largos/rf/StrokeSmote/rf_randomcv.pkl
Tipo de modelo: <class 'sklearn.model_selection._search.RandomizedSearchCV'>


In [ ]:
# ============================================================
# 2. Cargar datasets interno (original) y externo
# ============================================================

data_int_raw = pd.read_csv('/content/drive/MyDrive/Strokesdataset.csv')
data_ext_raw = pd.read_csv('/content/drive/MyDrive/synthetic_stroke_datasetv2.csv')

print("Dimensiones bruto interno:", data_int_raw.shape)
print("Dimensiones bruto externo:", data_ext_raw.shape)

# --- Comprobar posible fuga de pacientes por 'id' entre interno y externo ---

if 'id' in data_int_raw.columns and 'id' in data_ext_raw.columns:
    ids_int = data_int_raw['id'].unique()
    ids_ext = data_ext_raw['id'].unique()
    inter_ids = np.intersect1d(ids_int, ids_ext)
    print(f"Número de IDs comunes entre interno y externo: {len(inter_ids)}")
    if len(inter_ids) > 0:
        print("AVISO: hay pacientes que aparecen en ambos datasets (posible fuga).")
else:
    print("No se encontró columna 'id' en alguno de los datasets.")

Dimensiones bruto interno: (43400, 12)
Dimensiones bruto externo: (10000, 11)
No se encontró columna 'id' en alguno de los datasets.


In [ ]:
# ============================================================
# 3. Preprocesamiento consistente (igual que el usado en el entrenamiento)
# ============================================================

TARGET_COL = 'stroke'
CATEGORICAS = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

def limpiar_dataset(df, nombre='dataset'):
    """Elimina id (si existe), filas con NaN y resetea índice."""
    df = df.copy()
    if 'id' in df.columns:
        df = df.drop(columns=['id'])
    antes = len(df)
    df = df.dropna(how='any').reset_index(drop=True)
    despues = len(df)
    if antes != despues:
        print(f"[{nombre}] Filas eliminadas por NaN:", antes - despues)
    return df

def construir_Xy_con_dummies(df, nombre='dataset'):
    """
    Aplica el tratamiento de datos:
      - asume que TARGET_COL está en df
      - aplica get_dummies a columnas categóricas
    Devuelve X (features), y (target) y df_dum (dataframe completo con dummies).
    """
    df = df.copy()
    if TARGET_COL not in df.columns:
        raise ValueError(f"[{nombre}] No se encontró la columna objetivo '{TARGET_COL}'")

    # Asegurar que las categóricas existen (las que falten se ignoran)
    cols_cat_presentes = [c for c in CATEGORICAS if c in df.columns]

    df_dum = pd.get_dummies(df, columns=cols_cat_presentes)
    X = df_dum.drop(columns=[TARGET_COL])
    y = df_dum[TARGET_COL].astype(int)  # asegurar 0/1
    return X, y, df_dum

# --- Limpiar datasets brutos ---

df_int = limpiar_dataset(data_int_raw, nombre='interno')
df_ext = limpiar_dataset(data_ext_raw, nombre='externo')

# Guardar las columnas crudas para analizar work_type antes de dummies
if 'work_type' not in df_int.columns or 'work_type' not in df_ext.columns:
    raise ValueError("La columna 'work_type' debe estar presente en interno y externo.")

# --- Construir X, y para interno (solo para referenciar columnas/ drift) ---

X_int_full, y_int, df_int_dum = construir_Xy_con_dummies(df_int, nombre='interno')
print("\nDimensiones interno limpio (post dummies):", X_int_full.shape)
print("Prevalencia stroke interno limpio:", y_int.mean())

# Las columnas que el modelo espera deberían coincidir con las usadas en el entrenamiento interno
# Usar las columnas de X_int_full como referencia
feature_cols_model = list(X_int_full.columns)
print("\nNúmero de features (interno) usados para el modelo:", len(feature_cols_model))

[interno] Filas eliminadas por NaN: 14328

Dimensiones interno limpio (post dummies): (29072, 20)
Prevalencia stroke interno limpio: 0.018849752339020365

Número de features (interno) usados para el modelo: 20


In [ ]:
# ============================================================
# 4. Preparar dataset EXTERNO con las mismas columnas de entrada
# ============================================================

X_ext_raw, y_ext, df_ext_dum = construir_Xy_con_dummies(df_ext, nombre='externo')
print("Dimensiones externo limpio (post dummies, antes de reindex):", X_ext_raw.shape)
print("Prevalencia stroke externo limpio:", y_ext.mean())

# Reindexar las columnas del externo para que coincidan con las del modelo
X_ext = X_ext_raw.reindex(columns=feature_cols_model, fill_value=0)

print("Dimensiones externo final alineado al modelo:", X_ext.shape)

Dimensiones externo limpio (post dummies, antes de reindex): (10000, 20)
Prevalencia stroke externo limpio: 0.0191
Dimensiones externo final alineado al modelo: (10000, 20)


In [ ]:
# ============================================================
# 5. Funciones de métricas, bootstrap, permutation test
# ============================================================

def _y_score(est, X):
    """Devuelve score continuo para la clase positiva."""
    if hasattr(est, "predict_proba"):
        return est.predict_proba(X)[:, 1]
    if hasattr(est, "decision_function"):
        return est.decision_function(X)
    return est.predict(X)

def _rates(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    P, N = tp + fn, tn + fp

    def sdiv(a, b):
        return np.nan if b == 0 else a / b

    tpr = sdiv(tp, P)
    tnr = sdiv(tn, N)
    fpr = sdiv(fp, N)
    fnr = sdiv(fn, P)
    return dict(TP=tp, TN=tn, FP=fp, FN=fn, TPR=tpr, TNR=tnr, FPR=fpr, FNR=fnr)

def _metrics(y_true, y_pred, y_score):
    r = _rates(y_true, y_pred)
    bacc = balanced_accuracy_score(y_true, y_pred)
    tpr, tnr = r['TPR'], r['TNR']
    if np.isnan(tpr) or np.isnan(tnr):
        gmean = np.nan
    else:
        gmean = np.sqrt(max(0.0, tpr * tnr))
    mcc = matthews_corrcoef(y_true, y_pred)
    try:
        aupr = average_precision_score(y_true, y_score)
    except ValueError:
        aupr = np.nan
    return dict(BAcc=bacc, GMean=gmean, MCC=mcc, AUC_PR=aupr, **r)

def evaluar_dataset(estimator, X, y):
    """Evalúa un modelo en un dataset cualquiera con las métricas definidas."""
    y = pd.Series(y).reset_index(drop=True)
    X = X.reset_index(drop=True)
    y_pred = estimator.predict(X)
    y_score = _y_score(estimator, X)
    return pd.Series(_metrics(y, y_pred, y_score))

def bootstrap_ci_metrics(estimator, X, y, n_boot=1000, random_state=42):
    """Bootstrap para IC 95% de las métricas."""
    rng = np.random.RandomState(random_state)
    Xb = X.reset_index(drop=True)
    yb = pd.Series(y).reset_index(drop=True)
    n = len(Xb)

    registros = []
    for _ in range(n_boot):
        idx = rng.randint(0, n, n)
        X_sample = Xb.iloc[idx]
        y_sample = yb.iloc[idx]
        y_pred = estimator.predict(X_sample)
        y_score = _y_score(estimator, X_sample)
        registros.append(_metrics(y_sample, y_pred, y_score))

    df_boot = pd.DataFrame(registros)
    mean = df_boot.mean()
    ci_low = df_boot.quantile(0.025)
    ci_high = df_boot.quantile(0.975)

    resumen_ci = pd.concat([mean, ci_low, ci_high], axis=1)
    resumen_ci.columns = ['mean', 'ci_2.5%', 'ci_97.5%']
    return df_boot, resumen_ci

def permutation_test_mcc(estimator, X, y, n_perm=1000, random_state=42):
    """
    Sanity check con permutation test:
    Se barajan las etiquetas N veces y se calcula MCC.
    """
    rng = np.random.RandomState(random_state)
    Xp = X.reset_index(drop=True)
    y_true = pd.Series(y).reset_index(drop=True)

    # MCC observado
    y_pred = estimator.predict(Xp)
    y_score = _y_score(estimator, Xp)
    m_obs = _metrics(y_true, y_pred, y_score)['MCC']

    # Distribución nula: permutamos etiquetas
    m_perm = []
    for _ in range(n_perm):
        y_perm = rng.permutation(y_true.values)
        m = _metrics(y_perm, y_pred, y_score)['MCC']
        m_perm.append(m)

    m_perm = np.array(m_perm)
    p_val = (1 + np.sum(m_perm >= m_obs)) / (1 + n_perm)
    return m_obs, m_perm, p_val

def resumen_prevalencia_y_medias(X, y, nombre):
    prev = float(np.mean(y))
    medias = X.mean()
    resumen = pd.DataFrame({f'{nombre}_mean': medias})
    return prev, resumen

In [ ]:
# ============================================================
# 6. Evaluación global en dataset EXTERNO
# ============================================================

print("\n=== Evaluación GLOBAL en dataset EXTERNO ===")
metrics_ext = evaluar_dataset(rf_randomcv, X_ext, y_ext)
display(metrics_ext.to_frame(name='externo'))

print("\n=== IC 95% (bootstrap) en EXTERNO ===")
boot_ext, ci_ext = bootstrap_ci_metrics(rf_randomcv, X_ext, y_ext, n_boot=1000, random_state=42)
display(ci_ext[['mean', 'ci_2.5%', 'ci_97.5%']])

print("\n=== Permutation test MCC en EXTERNO ===")
mcc_obs_ext, mcc_perm_ext, p_val_ext = permutation_test_mcc(rf_randomcv, X_ext, y_ext, n_perm=1000, random_state=42)
print("MCC observado (externo):", mcc_obs_ext)
print("p-valor permutation test (externo):", p_val_ext)


=== Evaluación GLOBAL en dataset EXTERNO ===


,externo
BAcc,0.504267
GMean,0.102230
MCC,0.025518
AUC_PR,0.055176
TP,2.000000
TN,9790.000000
FP,19.000000
FN,189.000000
TPR,0.010471
TNR,0.998063



=== IC 95% (bootstrap) en EXTERNO ===


,mean,ci_2.5%,ci_97.5%
BAcc,0.504150,0.498778,0.512656
GMean,0.090180,0.000000,0.164165
MCC,0.024532,-0.006789,0.073181
AUC_PR,0.056443,0.046262,0.068343
TP,1.953000,0.000000,5.000000
TN,9790.053000,9763.000000,9816.025000
FP,19.081000,11.000000,28.000000
FN,188.913000,165.000000,216.000000
TPR,0.010245,0.000000,0.027027
TNR,0.998055,0.997149,0.998879



=== Permutation test MCC en EXTERNO ===
MCC observado (externo): 0.02551755696437879
p-valor permutation test (externo): 0.06393606393606394


In [ ]:
# ============================================================
# 7. ΔMCC: comparación validez interna vs externa
# ============================================================

# Aquí se introduce el MCC medio obtenido en la validación interna (CV / Nested CV)
# por ejemplo: MCC_CV_INTERNO = 0.962  (se ajusta este valor según los resultados reales)
MCC_CV_INTERNO = 0.9844  # <-- VALOR REAL DE MCC EN CV INTERNA

MCC_externo = metrics_ext['MCC']
delta_mcc_ext = MCC_externo - MCC_CV_INTERNO

print("\n=== Comparación interna vs externa (MCC) ===")
print("MCC (CV interna):", MCC_CV_INTERNO)
print("MCC (externo)   :", MCC_externo)
print("ΔMCC = MCC_externo - MCC_CV =", delta_mcc_ext)


=== Comparación interna vs externa (MCC) ===
MCC (CV interna): 0.9844
MCC (externo)   : 0.02551755696437879
ΔMCC = MCC_externo - MCC_CV = -0.9588824430356213


In [ ]:
# ============================================================
# 8. Análisis de drift entre interno y externo
# ============================================================

# Para drift de características para usar el espacio de X con dummies alineado
# Recalcular X_int alineado a las columnas del modelo
X_int_aligned = X_int_full.reindex(columns=feature_cols_model, fill_value=0)

print("\n=== Drift: prevalencia y distribuciones de características ===")

prev_int, feat_int = resumen_prevalencia_y_medias(X_int_aligned, y_int, 'interno')
prev_ext, feat_ext = resumen_prevalencia_y_medias(X_ext, y_ext, 'externo')

print("Prevalencia interno:", prev_int)
print("Prevalencia externo:", prev_ext)
print("ΔPrevalencia (externo - interno):", prev_ext - prev_int)

feat_drift = feat_int.join(feat_ext, how='outer')
feat_drift['delta'] = feat_drift['externo_mean'] - feat_drift['interno_mean']
feat_drift = feat_drift.sort_values('delta', key=lambda s: s.abs(), ascending=False)

print("\nTop 20 variables con mayor cambio de media (drift):")
display(feat_drift.head(20))


=== Drift: prevalencia y distribuciones de características ===
Prevalencia interno: 0.018849752339020365
Prevalencia externo: 0.0191
ΔPrevalencia (externo - interno): 0.0002502476609796342

Top 20 variables con mayor cambio de media (drift):


,interno_mean,externo_mean,delta
age,47.671746,38.373846,-9.297900
avg_glucose_level,106.403225,98.613063,-7.790161
bmi,30.054166,28.671765,-1.382401
work_type_children,0.021223,0.164400,0.143177
ever_married_Yes,0.746079,0.627500,-0.118579
ever_married_No,0.253921,0.372500,0.118579
work_type_Private,0.651968,0.549200,-0.102768
smoking_status_smokes,0.214158,0.147800,-0.066358
smoking_status_never smoked,0.541655,0.603500,0.061845
Residence_type_Urban,0.502029,0.547000,0.044971


In [ ]:
# ============================================================
# 9. Análisis por subgrupo demográfico sensible:
#    work_type == Self-employed
# ============================================================

print("\n=== Análisis por subgrupo: work_type Self-employed ===")

# Ojo: usar df_ext (sin dummies) para identificar subgrupo, y luego sincronizar los índices

mask_self_emp = df_ext['work_type'] == 'Self-employed'

X_ext_self = X_ext[mask_self_emp.values]
y_ext_self = y_ext[mask_self_emp.values]

X_ext_no_self = X_ext[~mask_self_emp.values]
y_ext_no_self = y_ext[~mask_self_emp.values]

print("Tamaño grupo Self-employed    :", X_ext_self.shape[0])
print("Tamaño grupo NO Self-employed :", X_ext_no_self.shape[0])

if X_ext_self.shape[0] > 0:
    metrics_ext_self = evaluar_dataset(rf_randomcv, X_ext_self, y_ext_self)
    print("\nMétricas en grupo Self-employed:")
    display(metrics_ext_self.to_frame(name='Self-employed'))
else:
    print("\n[AVISO] No hay filas con work_type = 'Self-employed' en el dataset externo.")

if X_ext_no_self.shape[0] > 0:
    metrics_ext_no_self = evaluar_dataset(rf_randomcv, X_ext_no_self, y_ext_no_self)
    print("\nMétricas en grupo NO Self-employed:")
    display(metrics_ext_no_self.to_frame(name='No Self-employed'))
else:
    print("\n[AVISO] No hay filas en el grupo No Self-employed en el dataset externo.")

# Diferencias en métricas clave entre grupos (si ambos existen)
if X_ext_self.shape[0] > 0 and X_ext_no_self.shape[0] > 0:
    dif_mcc = metrics_ext_self['MCC'] - metrics_ext_no_self['MCC']
    dif_tpr = metrics_ext_self['TPR'] - metrics_ext_no_self['TPR']
    dif_bacc = metrics_ext_self['BAcc'] - metrics_ext_no_self['BAcc']

    print("\n=== Diferencias de métricas Self-employed vs No Self-employed ===")
    print("ΔMCC (Self - NoSelf)      :", dif_mcc)
    print("ΔTPR / Sensibilidad       :", dif_tpr)
    print("ΔBalanced Accuracy (BAcc) :", dif_bacc)


=== Análisis por subgrupo: work_type Self-employed ===
Tamaño grupo Self-employed    : 1549
Tamaño grupo NO Self-employed : 8451

Métricas en grupo Self-employed:


,Self-employed
BAcc,0.498318
GMean,0.000000
MCC,-0.011717
AUC_PR,0.055324
TP,0.000000
TN,1481.000000
FP,5.000000
FN,63.000000
TPR,0.000000
TNR,0.996635



Métricas en grupo NO Self-employed:


,No Self-employed
BAcc,0.506971
GMean,0.124895
MCC,0.039174
AUC_PR,0.058626
TP,2.000000
TN,8309.000000
FP,14.000000
FN,126.000000
TPR,0.015625
TNR,0.998318



=== Diferencias de métricas Self-employed vs No Self-employed ===
ΔMCC (Self - NoSelf)      : -0.05089097896034621
ΔTPR / Sensibilidad       : -0.015625
ΔBalanced Accuracy (BAcc) : -0.008653825882047994
